In [1]:
import allel
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
#place the python file in the same folder containing all the real1, real2_part1, ... folders

def parse_to_df(folder='test', features='*', algos=['freebayes', 'mutect2', 'vardict', 'varscan']):
    '''
    reads all vcf.gz files corresponding to algos in the specified folder with the specified list of features
    and combines the read files into one dataframe with (CHROM, POS, REF) as index.
    '''
    if folder == 'test':
        dfs = [allel.vcf_to_dataframe(f'{folder}/{i}.vcf.gz', fields = features) for i in algos]
    else:
        dfs = [allel.vcf_to_dataframe(f'{folder}/{folder}-{i}.vcf.gz', fields = features) for i in algos]
    algo_dicts = dict(zip(algos, dfs))
    
    #some manipulations
    for i in algo_dicts:
        algo_dicts[i].set_index(keys=['CHROM', 'POS', 'REF'], inplace = True) #will be use as keys for later merging
        algo_dicts[i] = algo_dicts[i][algo_dicts[i]['is_snp']]    #obtain only SNPs
        algo_dicts[i].columns = [j + '_' + i for j in algo_dicts[i].columns]

    #combining the dfs
    edited_dfs = [algo_dicts[i] for i in algos]

    merged = reduce(lambda left, right: pd.merge(left, right,
                                            how = 'outer',
                                            left_index=True, right_index=True,
                                            suffixes = ('', '')), edited_dfs)

    merged.columns = sorted(merged.columns)

    return merged


In [251]:
# parse_to_df function takes too long to run so I parse individually 
# try with rand subset of features 
varscan_features = ['CHROM','POS','REF','ALT_1', 'SSC','SPV','is_snp']
real1_varscan_sub = allel.vcf_to_dataframe("syn1/syn1-varscan.vcf.gz", fields = varscan_features)

freebayes_features = ['CHROM','POS','REF','ALT_1', 'MQMR','is_snp']
real1_freebayes_sub = allel.vcf_to_dataframe("syn1/syn1-freebayes.vcf.gz", fields = freebayes_features)

mutect2_features = ['CHROM','POS','REF','ALT_1', 'mQ','is_snp']
real1_mutect2_sub = allel.vcf_to_dataframe("syn1/syn1-mutect2.vcf.gz", fields = mutect2_features)

vardict_features = ['CHROM','POS','REF','ALT_1', 'SSF','MSI','is_snp']
real1_vardict_sub = allel.vcf_to_dataframe("syn1/syn1-vardict.vcf.gz", fields = vardict_features)

/Users/ting/opt/anaconda3/lib/python3.9/site-packages/allel/io/vcf_read.py:1240: UserWarning: 'ALT_1' INFO header not found
  warnings.warn('%r INFO header not found' % name)
/Users/ting/opt/anaconda3/lib/python3.9/site-packages/allel/io/vcf_read.py:1454: UserWarning: no type for field 'variants/ALT_1', assuming object
  warnings.warn('no type for field %r, assuming %s' % (f, normed_types[f]))
/Users/ting/opt/anaconda3/lib/python3.9/site-packages/allel/io/vcf_read.py:1564: UserWarning: no number for field 'variants/ALT_1', assuming 1
  warnings.warn('no number for field %r, assuming 1' % f)
/Users/ting/opt/anaconda3/lib/python3.9/site-packages/allel/io/vcf_read.py:1240: UserWarning: 'mQ' INFO header not found
  warnings.warn('%r INFO header not found' % name)
/Users/ting/opt/anaconda3/lib/python3.9/site-packages/allel/io/vcf_read.py:1454: UserWarning: no type for field 'variants/mQ', assuming object
  warnings.warn('no type for field %r, assuming %s' % (f, normed_types[f]))
/Users/ting

In [252]:
# subsetting for snp = True 
varscan_sub = real1_varscan_sub[real1_varscan_sub.is_snp == True]
freebayes_sub = real1_freebayes_sub[real1_freebayes_sub.is_snp == True]
mutect2_sub = real1_mutect2_sub[real1_mutect2_sub.is_snp == True]
vardict_sub = real1_vardict_sub[real1_vardict_sub.is_snp == True]

In [253]:
# Gradient Boosting Decision Trees
# popular algorithms like XGboost and Catboost are examples of using the gradient boosting framework 
# unlike random forests, the decision trees in gradient boosting are built additively; each decision tree is built one after another
# each new treee is built to improve on deficiencies of the previous trees and this concept is called boosting 
# gradient of gradient boosting comes from minimising the gradient of the loss function 

In [254]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
from numpy import nan

In [255]:
lst_dfs = [varscan_sub,freebayes_sub,mutect2_sub,vardict_sub]
suffix = ['vs','fb','m2','vd']
keep_same = {'CHROM', 'POS'}
i =0 
for df in lst_dfs:
    df.columns = ['{}{}'.format(c, '' if c in keep_same else '_'+suffix[i]) for c in df.columns]
    i += 1
lst_dfs

[          CHROM     POS REF_vs  ALT_1_vs  SSC_vs    SPV_vs  is_snp_vs
 3             1   10247      T       NaN    11.0  0.070191       True
 4             1   10248      A       NaN     8.0  0.152890       True
 6             1   10257      A       NaN    11.0  0.068939       True
 11            1   12783      G       NaN    16.0  0.024513       True
 12            1   12807      C       NaN     6.0  0.239270       True
 ..          ...     ...    ...       ...     ...       ...        ...
 229  GL000192.1  546648      C       NaN     0.0  0.896130       True
 230  GL000192.1  547087      T       NaN     4.0  0.376560       True
 231  GL000192.1  547102      C       NaN     1.0  0.728090       True
 232  GL000192.1  547218      C       NaN     3.0  0.469670       True
 233  GL000192.1  547406      G       NaN     2.0  0.520770       True
 
 [4119533 rows x 7 columns],
             CHROM     POS REF_fb  ALT_1_fb    MQMR_fb  is_snp_fb
 1               1   10583      G       NaN  41.000

In [257]:
merged_df = reduce(lambda left, right: pd.merge(left, right,on =['CHROM', 'POS'],
                                            how = 'outer', suffixes = ('', '')),lst_dfs)
merged_df = merged_df.drop(['is_snp_vd','is_snp_fb','is_snp_m2','is_snp_vs'], axis=1)
merged_df.to_csv("syn1_mergered_df.csv")

In [258]:
from sklearn import datasets
import xgboost as xgb 
from xgboost import XGBClassifier

In [262]:
##  function to get y labels
truth_labels = pd.read_csv("syn1/syn1_truth.bed", sep = "\t", names = ['Chromo', 'start', 'end'])
print(list(set(truth_labels.start == truth_labels.end) )) # the start and end position are the same 
truth_labels = truth_labels[['Chromo', 'start']]
truth_labels['truth'] = 1
sub_truth= truth_labels.rename(columns = {'Chromo':'CHROM', 'start':'POS'})

[True]


In [263]:
# combine dataset 
combined = merged_df.merge(sub_truth, on=['CHROM','POS'], how = 'left' )
combined['truth'].fillna(0, inplace = True)

In [264]:
X = combined[combined.columns[~combined.columns.isin(['truth','POS','CHROM'])]]

y = combined['truth'] 

In [265]:
# ordinal encoding for REF and ALT
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(X)
new_X = enc.transform(X)

In [266]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.2)

In [267]:
model = XGBClassifier(eval_metric='rmse')

In [268]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='rmse', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [269]:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

/Users/ting/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy=None, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=0, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', predictor=None, ...)

In [270]:
y_pred = model.predict(X_test)

In [271]:
# evaluate model performance # for syn1 dataset 
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[926684     45]
 [    45    664]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    926729
         1.0       0.94      0.94      0.94       709

    accuracy                           1.00    927438
   macro avg       0.97      0.97      0.97    927438
weighted avg       1.00      1.00      1.00    927438



In [246]:
# evaluate model performance # for real1 dataset 
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[930908     27]
 [    42    227]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    930935
         1.0       0.89      0.84      0.87       269

    accuracy                           1.00    931204
   macro avg       0.95      0.92      0.93    931204
weighted avg       1.00      1.00      1.00    931204

